# Proc_1

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# 修理データの読み込み
df_repair = pd.read_pickle('df_repair.pkl')

# データの前処理
# dateカラムを年月に変換
df_repair['repair_year_month'] = df_repair['date'].dt.to_period('M')

# prod_monthカラムを年月に変換（originalのテストデータがperiodだったので修正）
#df_repair['prod_year_month'] = df_repair['prod_month'].dt.to_period('M')
df_repair['prod_year_month'] = df_repair['prod_month'].copy()

print("データの形状:", df_repair.shape)
print("データの概要:")
print(df_repair.head())

# 1. 部品使用数の集計
print("\n=== 部品使用数の集計開始 ===")

# Area, HEAD, prod_month, Model, parts_noでグループ化して部品使用数を集計
parts_usage = df_repair.groupby(['Area', 'HEAD', 'prod_year_month', 'Model', 'parts_no', 'repair_year_month']).size().reset_index(name='parts_count')

# 各グループの最初の修理発生月を取得
first_repair_month = parts_usage.groupby(['Area', 'HEAD', 'prod_year_month', 'Model', 'parts_no'])['repair_year_month'].min().reset_index()
first_repair_month.rename(columns={'repair_year_month': 'first_repair_month'}, inplace=True)

# 全体の修理期間を取得
min_repair_month = df_repair['repair_year_month'].min()
max_repair_month = df_repair['repair_year_month'].max()

# 全期間の年月リストを作成
all_months = pd.period_range(start=min_repair_month, end=max_repair_month, freq='M')

# 各グループに対して全期間の行を作成
parts_usage_complete = []

for _, group in first_repair_month.groupby(['Area', 'HEAD', 'prod_year_month', 'Model', 'parts_no']):
    area, head, prod_month, model, parts_no = group.iloc[0][['Area', 'HEAD', 'prod_year_month', 'Model', 'parts_no']]
    first_month = group.iloc[0]['first_repair_month']
    
    # 最初の修理月以降の全期間を対象とする
    target_months = [m for m in all_months if m >= first_month]
    
    for month in target_months:
        parts_usage_complete.append({
            'Area': area,
            'HEAD': head,
            'prod_year_month': prod_month,
            'Model': model,
            'parts_no': parts_no,
            'repair_year_month': month,
            'parts_count': 0  # デフォルトは0
        })

# DataFrameに変換
df_parts_complete = pd.DataFrame(parts_usage_complete)

# 実際の部品使用数をマージ
df_parts_final = df_parts_complete.merge(
    parts_usage[['Area', 'HEAD', 'prod_year_month', 'Model', 'parts_no', 'repair_year_month', 'parts_count']],
    on=['Area', 'HEAD', 'prod_year_month', 'Model', 'parts_no', 'repair_year_month'],
    how='left',
    suffixes=('', '_actual')
)

# 実際の値がある場合はそれを使用、ない場合は0
df_parts_final['parts_count'] = df_parts_final['parts_count_actual'].fillna(df_parts_final['parts_count'])
df_parts_final = df_parts_final.drop('parts_count_actual', axis=1)

# 累積部品使用数を計算
df_parts_final = df_parts_final.sort_values(['Area', 'HEAD', 'prod_year_month', 'Model', 'parts_no', 'repair_year_month'])
df_parts_final['cumulative_parts_count'] = df_parts_final.groupby(['Area', 'HEAD', 'prod_year_month', 'Model', 'parts_no'])['parts_count'].cumsum()

print(f"部品使用数データの形状: {df_parts_final.shape}")

# 2. 修理数の集計
print("\n=== 修理数の集計開始 ===")

# IF_IDでユニークな修理数を集計
repair_unique = df_repair.drop_duplicates(subset=['Area', 'HEAD', 'prod_year_month', 'Model', 'repair_year_month', 'IF_ID'])
repair_counts = repair_unique.groupby(['Area', 'HEAD', 'prod_year_month', 'Model', 'repair_year_month']).size().reset_index(name='repair_count')

# 各グループの最初の修理発生月を取得
first_repair_month_repair = repair_counts.groupby(['Area', 'HEAD', 'prod_year_month', 'Model'])['repair_year_month'].min().reset_index()
first_repair_month_repair.rename(columns={'repair_year_month': 'first_repair_month'}, inplace=True)

# 各グループに対して全期間の行を作成
repair_counts_complete = []

for _, group in first_repair_month_repair.groupby(['Area', 'HEAD', 'prod_year_month', 'Model']):
    area, head, prod_month, model = group.iloc[0][['Area', 'HEAD', 'prod_year_month', 'Model']]
    first_month = group.iloc[0]['first_repair_month']
    
    # 最初の修理月以降の全期間を対象とする
    target_months = [m for m in all_months if m >= first_month]
    
    for month in target_months:
        repair_counts_complete.append({
            'Area': area,
            'HEAD': head,
            'prod_year_month': prod_month,
            'Model': model,
            'repair_year_month': month,
            'repair_count': 0  # デフォルトは0
        })

# DataFrameに変換
df_repair_complete = pd.DataFrame(repair_counts_complete)

# 実際の修理数をマージ
df_repair_final = df_repair_complete.merge(
    repair_counts[['Area', 'HEAD', 'prod_year_month', 'Model', 'repair_year_month', 'repair_count']],
    on=['Area', 'HEAD', 'prod_year_month', 'Model', 'repair_year_month'],
    how='left',
    suffixes=('', '_actual')
)

# 実際の値がある場合はそれを使用、ない場合は0
df_repair_final['repair_count'] = df_repair_final['repair_count_actual'].fillna(df_repair_final['repair_count'])
df_repair_final = df_repair_final.drop('repair_count_actual', axis=1)

# 累積修理数を計算
df_repair_final = df_repair_final.sort_values(['Area', 'HEAD', 'prod_year_month', 'Model', 'repair_year_month'])
df_repair_final['cumulative_repair_count'] = df_repair_final.groupby(['Area', 'HEAD', 'prod_year_month', 'Model'])['repair_count'].cumsum()

print(f"修理数データの形状: {df_repair_final.shape}")

# 3. データの保存
print("\n=== データの保存 ===")

# 部品使用数データの保存
parts_filename = 'parts_usage_monthly_cumulative.pkl'
df_parts_final.to_pickle(parts_filename)
print(f"部品使用数データを {parts_filename} に保存しました")

# 修理数データの保存
repair_filename = 'repair_counts_monthly_cumulative.pkl'
df_repair_final.to_pickle(repair_filename)
print(f"修理数データを {repair_filename} に保存しました")

# 4. 結果の確認
print("\n=== 結果の確認 ===")
print("部品使用数データのサンプル:")
print(df_parts_final.head(10))
print("\n修理数データのサンプル:")
print(df_repair_final.head(10))

print(f"\n処理完了!")
print(f"部品使用数データ: {df_parts_final.shape[0]} 行")
print(f"修理数データ: {df_repair_final.shape[0]} 行")

データの形状: (1527, 9)
データの概要:
        date prod_month HEAD Model Area  parts_no     IF_ID repair_year_month  \
0 2023-01-01    2022-07   10  M200  EUR  20000012  10000019           2023-01   
1 2023-01-04    2021-10   01  M200   CN  20000010  10000017           2023-01   
2 2023-01-08    2022-02   23  M250  EUR  25000018  10000182           2023-01   
3 2023-01-08    2022-02   23  M250  EUR  25000013  10000182           2023-01   
4 2023-01-10    2021-10   01  M200   JP  20000010  10000282           2023-01   

  prod_year_month  
0         2022-07  
1         2021-10  
2         2022-02  
3         2022-02  
4         2021-10  

=== 部品使用数の集計開始 ===
部品使用数データの形状: (16763, 8)

=== 修理数の集計開始 ===
修理数データの形状: (3393, 7)

=== データの保存 ===
部品使用数データを parts_usage_monthly_cumulative.pkl に保存しました
修理数データを repair_counts_monthly_cumulative.pkl に保存しました

=== 結果の確認 ===
部品使用数データのサンプル:
  Area HEAD prod_year_month Model  parts_no repair_year_month  parts_count  \
0   CN   01         2021-10  M200  20000001           

# Proc_2

In [6]:
import pandas as pd
import numpy as np
from datetime import datetime

def calculate_modified_zscore(data):
    """修正Z-Score（MAD基準）を計算する関数"""
    median = np.median(data)
    mad = np.median(np.abs(data - median))
    # MADが0の場合の処理
    if mad == 0:
        return np.zeros(len(data))
    else:
        return 0.6745 * (data - median) / mad

def calculate_rolling_zscore(group, value_col, date_col, window_months=12):
    """過去12ヶ月の移動窓を使った修正Z-Scoreを計算する関数"""
    group = group.sort_values(date_col).reset_index(drop=True)
    rolling_zscores = []
    
    for i in range(len(group)):
        current_date = group.iloc[i][date_col]
        # 現在の日付から過去12ヶ月のデータを取得
        start_date = current_date - 11  # 11ヶ月前から現在まで（計12ヶ月）
        
        # 過去12ヶ月のデータを抽出
        mask = (group[date_col] >= start_date) & (group[date_col] <= current_date)
        window_data = group.loc[mask, value_col].values
        
        if len(window_data) >= 3:  # 最低3データポイントが必要
            # 現在の値
            current_value = group.iloc[i][value_col]
            # 過去12ヶ月のデータで修正Z-Scoreを計算
            median = np.median(window_data)
            mad = np.median(np.abs(window_data - median))
            if mad == 0:
                zscore = 0
            else:
                zscore = 0.6745 * (current_value - median) / mad
            rolling_zscores.append(zscore)
        else:
            rolling_zscores.append(np.nan)  # データが不足している場合
    
    return pd.Series(rolling_zscores, index=group.index)

# データの読み込み
print("=== データの読み込み ===")
df_parts = pd.read_pickle('parts_usage_monthly_cumulative.pkl')
df_repair = pd.read_pickle('repair_counts_monthly_cumulative.pkl')
df_sales = pd.read_pickle('df_sales.pkl')

print(f"部品使用数データ: {df_parts.shape}")
print(f"修理数データ: {df_repair.shape}")
print(f"販売データ: {df_sales.shape}")

# 販売データの前処理
df_sales['sales_year_month'] = df_sales['date'].dt.to_period('M')

# 同じdate、Model、Areaで複数のQT_ALLがある場合は合計する
df_sales_aggregated = df_sales.groupby(['date', 'Model', 'Area', 'sales_year_month']).agg({
    'QT_ALL': 'sum'
}).reset_index()

# 各グループの最初の販売発生月を取得
first_sales_month = df_sales_aggregated.groupby(['Area', 'Model'])['sales_year_month'].min().reset_index()
first_sales_month.rename(columns={'sales_year_month': 'first_sales_month'}, inplace=True)

# 全体の販売期間を取得
min_sales_month = df_sales_aggregated['sales_year_month'].min()
max_sales_month = df_sales_aggregated['sales_year_month'].max()

# 全期間の年月リストを作成
all_sales_months = pd.period_range(start=min_sales_month, end=max_sales_month, freq='M')

# 各グループに対して全期間の行を作成
sales_complete = []

for _, group in first_sales_month.groupby(['Area', 'Model']):
    area, model = group.iloc[0][['Area', 'Model']]
    first_month = group.iloc[0]['first_sales_month']
    
    # 最初の販売月以降の全期間を対象とする
    target_months = [m for m in all_sales_months if m >= first_month]
    
    for month in target_months:
        sales_complete.append({
            'Area': area,
            'Model': model,
            'sales_year_month': month,
            'QT_ALL': 0  # デフォルトは0
        })

# DataFrameに変換
df_sales_complete = pd.DataFrame(sales_complete)

# 実際の販売数をマージ
df_sales_final = df_sales_complete.merge(
    df_sales_aggregated[['Area', 'Model', 'sales_year_month', 'QT_ALL']],
    on=['Area', 'Model', 'sales_year_month'],
    how='left',
    suffixes=('', '_actual')
)

# 実際の値がある場合はそれを使用、ない場合は0
df_sales_final['QT_ALL'] = df_sales_final['QT_ALL_actual'].fillna(df_sales_final['QT_ALL'])
df_sales_final = df_sales_final.drop('QT_ALL_actual', axis=1)

# 累計販売台数を正しく計算（Area、Modelごとに年月順でソートして累計）
df_sales_cumsum = df_sales_final.sort_values(['Area', 'Model', 'sales_year_month']).groupby(['Area', 'Model']).apply(
    lambda x: x.assign(cumulative_sales=x['QT_ALL'].cumsum())
).reset_index(drop=True)

print(f"販売データ集計後: {df_sales_cumsum.shape}")
print("販売データのサンプル（累計確認用）:")
sample_data = df_sales_cumsum[(df_sales_cumsum['Model'] == 'M100') & (df_sales_cumsum['Area'] == 'JP')].head(10)
print(sample_data[['sales_year_month', 'QT_ALL', 'cumulative_sales']])

print("\n=== 1. 各機種の各部品の毎月の分析 ===")

# 機種・部品レベルでの集計（地域・製造年月は無視）
parts_model_level = df_parts.groupby(['Model', 'parts_no', 'repair_year_month']).agg({
    'parts_count': 'sum',
    'cumulative_parts_count': 'sum'
}).reset_index()

repair_model_level = df_repair.groupby(['Model', 'repair_year_month']).agg({
    'repair_count': 'sum',
    'cumulative_repair_count': 'sum'
}).reset_index()

# 販売データを年月レベルで集計
sales_model_level = df_sales_cumsum.groupby(['Model', 'sales_year_month']).agg({
    'QT_ALL': 'sum',
    'cumulative_sales': 'sum'
}).reset_index()

# データをマージ
df_analysis1 = parts_model_level.merge(
    repair_model_level, 
    left_on=['Model', 'repair_year_month'], 
    right_on=['Model', 'repair_year_month'], 
    how='left'
)

df_analysis1 = df_analysis1.merge(
    sales_model_level,
    left_on=['Model', 'repair_year_month'],
    right_on=['Model', 'sales_year_month'],
    how='left'
)

# 使用率の計算
df_analysis1['parts_usage_rate'] = df_analysis1['parts_count'] / df_analysis1['repair_count'].replace(0, np.nan)  # 部品使用率
df_analysis1['cumulative_parts_usage_rate'] = df_analysis1['cumulative_parts_count'] / df_analysis1['cumulative_repair_count'].replace(0, np.nan)  # 累計部品使用率
df_analysis1['parts_failure_rate'] = df_analysis1['cumulative_parts_count'] / df_analysis1['cumulative_sales'].replace(0, np.nan)  # 部品故障率

# 修正Z-Scoreの計算（全期間 + 過去12ヶ月移動窓）
df_analysis1['parts_count_zscore'] = df_analysis1.groupby(['Model', 'parts_no'])['parts_count'].transform(calculate_modified_zscore)
df_analysis1['cumulative_parts_count_zscore'] = df_analysis1.groupby(['Model', 'parts_no'])['cumulative_parts_count'].transform(calculate_modified_zscore)
df_analysis1['parts_usage_rate_zscore'] = df_analysis1.groupby(['Model', 'parts_no'])['parts_usage_rate'].transform(calculate_modified_zscore)
df_analysis1['cumulative_parts_usage_rate_zscore'] = df_analysis1.groupby(['Model', 'parts_no'])['cumulative_parts_usage_rate'].transform(calculate_modified_zscore)
df_analysis1['parts_failure_rate_zscore'] = df_analysis1.groupby(['Model', 'parts_no'])['parts_failure_rate'].transform(calculate_modified_zscore)

# 過去12ヶ月移動窓での修正Z-Score計算（修正版）
rolling_results_1a = []
for name, group in df_analysis1.groupby(['Model', 'parts_no']):
    group_sorted = group.sort_values('repair_year_month').reset_index()
    parts_count_12m = calculate_rolling_zscore(group_sorted, 'parts_count', 'repair_year_month')
    cumulative_parts_count_12m = calculate_rolling_zscore(group_sorted, 'cumulative_parts_count', 'repair_year_month')
    
    for i, (idx, parts_zscore, cum_zscore) in enumerate(zip(group_sorted['index'], parts_count_12m, cumulative_parts_count_12m)):
        rolling_results_1a.append({
            'index': idx,
            'parts_count_zscore_12m': parts_zscore,
            'cumulative_parts_count_zscore_12m': cum_zscore
        })

rolling_df_1a = pd.DataFrame(rolling_results_1a).set_index('index')
df_analysis1 = df_analysis1.join(rolling_df_1a)

print(f"分析1データ形状: {df_analysis1.shape}")

print("\n=== 2. 各機種の各部品の地域別分析 ===")

# 機種・部品・地域レベルでの集計
parts_area_level = df_parts.groupby(['Model', 'parts_no', 'Area', 'repair_year_month']).agg({
    'parts_count': 'sum',
    'cumulative_parts_count': 'sum'
}).reset_index()

repair_area_level = df_repair.groupby(['Model', 'Area', 'repair_year_month']).agg({
    'repair_count': 'sum',
    'cumulative_repair_count': 'sum'
}).reset_index()

# 地域別販売データ
sales_area_level = df_sales_cumsum.groupby(['Model', 'Area', 'sales_year_month']).agg({
    'QT_ALL': 'sum',
    'cumulative_sales': 'sum'
}).reset_index()

# データをマージ
df_analysis2 = parts_area_level.merge(
    repair_area_level, 
    on=['Model', 'Area', 'repair_year_month'], 
    how='left'
)

df_analysis2 = df_analysis2.merge(
    sales_area_level,
    left_on=['Model', 'Area', 'repair_year_month'],
    right_on=['Model', 'Area', 'sales_year_month'],
    how='left'
)

# 使用率の計算
df_analysis2['parts_usage_rate'] = df_analysis2['parts_count'] / df_analysis2['repair_count'].replace(0, np.nan)  # 部品使用率
df_analysis2['cumulative_parts_usage_rate'] = df_analysis2['cumulative_parts_count'] / df_analysis2['cumulative_repair_count'].replace(0, np.nan)  # 累計部品使用率
df_analysis2['parts_failure_rate'] = df_analysis2['cumulative_parts_count'] / df_analysis2['cumulative_sales'].replace(0, np.nan)  # 部品故障率

# 修正Z-Scoreの計算（全期間 + 過去12ヶ月移動窓）
df_analysis2['parts_count_zscore'] = df_analysis2.groupby(['Model', 'parts_no', 'Area'])['parts_count'].transform(calculate_modified_zscore)
df_analysis2['cumulative_parts_count_zscore'] = df_analysis2.groupby(['Model', 'parts_no', 'Area'])['cumulative_parts_count'].transform(calculate_modified_zscore)
df_analysis2['parts_usage_rate_zscore'] = df_analysis2.groupby(['Model', 'parts_no', 'Area'])['parts_usage_rate'].transform(calculate_modified_zscore)
df_analysis2['cumulative_parts_usage_rate_zscore'] = df_analysis2.groupby(['Model', 'parts_no', 'Area'])['cumulative_parts_usage_rate'].transform(calculate_modified_zscore)
df_analysis2['parts_failure_rate_zscore'] = df_analysis2.groupby(['Model', 'parts_no', 'Area'])['parts_failure_rate'].transform(calculate_modified_zscore)

# 過去12ヶ月移動窓での修正Z-Score計算（修正版）
rolling_results_2a = []
for name, group in df_analysis2.groupby(['Model', 'parts_no', 'Area']):
    group_sorted = group.sort_values('repair_year_month').reset_index()
    parts_count_12m = calculate_rolling_zscore(group_sorted, 'parts_count', 'repair_year_month')
    cumulative_parts_count_12m = calculate_rolling_zscore(group_sorted, 'cumulative_parts_count', 'repair_year_month')
    
    for i, (idx, parts_zscore, cum_zscore) in enumerate(zip(group_sorted['index'], parts_count_12m, cumulative_parts_count_12m)):
        rolling_results_2a.append({
            'index': idx,
            'parts_count_zscore_12m': parts_zscore,
            'cumulative_parts_count_zscore_12m': cum_zscore
        })

rolling_df_2a = pd.DataFrame(rolling_results_2a).set_index('index')
df_analysis2 = df_analysis2.join(rolling_df_2a)

print(f"分析2データ形状: {df_analysis2.shape}")

print("\n=== 3. 各機種の各部品の製造年月別分析 ===")

# prod_monthがNaNでないデータのみを使用
df_parts_prod = df_parts[df_parts['prod_year_month'].notna()].copy()
df_repair_prod = df_repair[df_repair['prod_year_month'].notna()].copy()

# 機種・部品・製造年月レベルでの集計
parts_prod_level = df_parts_prod.groupby(['Model', 'parts_no', 'prod_year_month', 'repair_year_month']).agg({
    'parts_count': 'sum',
    'cumulative_parts_count': 'sum'
}).reset_index()

repair_prod_level = df_repair_prod.groupby(['Model', 'prod_year_month', 'repair_year_month']).agg({
    'repair_count': 'sum',
    'cumulative_repair_count': 'sum'
}).reset_index()

# データをマージ
df_analysis3 = parts_prod_level.merge(
    repair_prod_level, 
    on=['Model', 'prod_year_month', 'repair_year_month'], 
    how='left'
)

# 製造年月からの経過月を計算
df_analysis3['months_from_production'] = (df_analysis3['repair_year_month'] - df_analysis3['prod_year_month']).apply(lambda x: x.n if pd.notna(x) else np.nan)

# 使用率の計算
df_analysis3['parts_usage_rate'] = df_analysis3['parts_count'] / df_analysis3['repair_count'].replace(0, np.nan)  # 部品使用率
df_analysis3['cumulative_parts_usage_rate'] = df_analysis3['cumulative_parts_count'] / df_analysis3['cumulative_repair_count'].replace(0, np.nan)  # 累計部品使用率

# 修正Z-Scoreの計算（経過月数を考慮 + 過去12ヶ月移動窓）
df_analysis3['parts_count_zscore'] = df_analysis3.groupby(['Model', 'parts_no', 'months_from_production'])['parts_count'].transform(calculate_modified_zscore)
df_analysis3['cumulative_parts_count_zscore'] = df_analysis3.groupby(['Model', 'parts_no', 'months_from_production'])['cumulative_parts_count'].transform(calculate_modified_zscore)
df_analysis3['parts_usage_rate_zscore'] = df_analysis3.groupby(['Model', 'parts_no', 'months_from_production'])['parts_usage_rate'].transform(calculate_modified_zscore)
df_analysis3['cumulative_parts_usage_rate_zscore'] = df_analysis3.groupby(['Model', 'parts_no', 'months_from_production'])['cumulative_parts_usage_rate'].transform(calculate_modified_zscore)

# 過去12ヶ月移動窓での修正Z-Score計算（修正版）
rolling_results_3a = []
for name, group in df_analysis3.groupby(['Model', 'parts_no', 'prod_year_month']):
    group_sorted = group.sort_values('repair_year_month').reset_index()
    parts_count_12m = calculate_rolling_zscore(group_sorted, 'parts_count', 'repair_year_month')
    cumulative_parts_count_12m = calculate_rolling_zscore(group_sorted, 'cumulative_parts_count', 'repair_year_month')
    
    for i, (idx, parts_zscore, cum_zscore) in enumerate(zip(group_sorted['index'], parts_count_12m, cumulative_parts_count_12m)):
        rolling_results_3a.append({
            'index': idx,
            'parts_count_zscore_12m': parts_zscore,
            'cumulative_parts_count_zscore_12m': cum_zscore
        })

rolling_df_3a = pd.DataFrame(rolling_results_3a).set_index('index')
df_analysis3 = df_analysis3.join(rolling_df_3a)

print(f"分析3データ形状: {df_analysis3.shape}")

print("\n=== 4. 各機種の各部品の地域別かつ製造年月別分析 ===")

# 機種・部品・地域・製造年月レベルでの集計
parts_area_prod_level = df_parts_prod.groupby(['Model', 'parts_no', 'Area', 'prod_year_month', 'repair_year_month']).agg({
    'parts_count': 'sum',
    'cumulative_parts_count': 'sum'
}).reset_index()

repair_area_prod_level = df_repair_prod.groupby(['Model', 'Area', 'prod_year_month', 'repair_year_month']).agg({
    'repair_count': 'sum',
    'cumulative_repair_count': 'sum'
}).reset_index()

# データをマージ
df_analysis4 = parts_area_prod_level.merge(
    repair_area_prod_level, 
    on=['Model', 'Area', 'prod_year_month', 'repair_year_month'], 
    how='left'
)

# 製造年月からの経過月を計算
df_analysis4['months_from_production'] = (df_analysis4['repair_year_month'] - df_analysis4['prod_year_month']).apply(lambda x: x.n if pd.notna(x) else np.nan)

# 使用率の計算
df_analysis4['parts_usage_rate'] = df_analysis4['parts_count'] / df_analysis4['repair_count'].replace(0, np.nan)  # 部品使用率
df_analysis4['cumulative_parts_usage_rate'] = df_analysis4['cumulative_parts_count'] / df_analysis4['cumulative_repair_count'].replace(0, np.nan)  # 累計部品使用率

# 修正Z-Scoreの計算（経過月数を考慮 + 過去12ヶ月移動窓）
df_analysis4['parts_count_zscore'] = df_analysis4.groupby(['Model', 'parts_no', 'Area', 'months_from_production'])['parts_count'].transform(calculate_modified_zscore)
df_analysis4['cumulative_parts_count_zscore'] = df_analysis4.groupby(['Model', 'parts_no', 'Area', 'months_from_production'])['cumulative_parts_count'].transform(calculate_modified_zscore)
df_analysis4['parts_usage_rate_zscore'] = df_analysis4.groupby(['Model', 'parts_no', 'Area', 'months_from_production'])['parts_usage_rate'].transform(calculate_modified_zscore)
df_analysis4['cumulative_parts_usage_rate_zscore'] = df_analysis4.groupby(['Model', 'parts_no', 'Area', 'months_from_production'])['cumulative_parts_usage_rate'].transform(calculate_modified_zscore)

# 過去12ヶ月移動窓での修正Z-Score計算
df_analysis4['parts_count_zscore_12m'] = df_analysis4.groupby(['Model', 'parts_no', 'Area', 'prod_year_month']).apply(
    lambda x: calculate_rolling_zscore(x, 'parts_count', 'repair_year_month')
).reset_index(level=[0, 1, 2, 3], drop=True)

df_analysis4['cumulative_parts_count_zscore_12m'] = df_analysis4.groupby(['Model', 'parts_no', 'Area', 'prod_year_month']).apply(
    lambda x: calculate_rolling_zscore(x, 'cumulative_parts_count', 'repair_year_month')
).reset_index(level=[0, 1, 2, 3], drop=True)

print(f"分析4データ形状: {df_analysis4.shape}")

print("\n=== データの保存 ===")

# 各分析結果を保存
df_analysis1.to_pickle('analysis1_model_parts_zscore.pkl')
df_analysis2.to_pickle('analysis2_area_parts_zscore.pkl')
df_analysis3.to_pickle('analysis3_production_month_zscore.pkl')
df_analysis4.to_pickle('analysis4_area_production_month_zscore.pkl')

print("分析1（機種・部品レベル）: analysis1_model_parts_zscore.pkl")
print("分析2（地域別）: analysis2_area_parts_zscore.pkl")
print("分析3（製造年月別）: analysis3_production_month_zscore.pkl")
print("分析4（地域・製造年月別）: analysis4_area_production_month_zscore.pkl")

print("\n=== 結果の確認 ===")
print("分析1のサンプル:")
print(df_analysis1.head())
print(f"\n分析1のカラム: {list(df_analysis1.columns)}")

print("\n分析2のサンプル:")
print(df_analysis2.head())
print(f"\n分析2のカラム: {list(df_analysis2.columns)}")

print("\n分析3のサンプル:")
print(df_analysis3.head())
print(f"\n分析3のカラム: {list(df_analysis3.columns)}")

print("\n分析4のサンプル:")
print(df_analysis4.head())
print(f"\n分析4のカラム: {list(df_analysis4.columns)}")

print("\n=== 処理完了 ===")
print("全ての分析が完了し、データが保存されました。")

=== データの読み込み ===
部品使用数データ: (16763, 8)
修理数データ: (3393, 7)
販売データ: (464, 4)
販売データ集計後: (464, 6)
販売データのサンプル（累計確認用）:
    sales_year_month  QT_ALL  cumulative_sales
232          2023-01      89                89
233          2023-02      85               174
234          2023-03     103               277
235          2023-04     126               403
236          2023-05     112               515
237          2023-06     138               653
238          2023-07     119               772
239          2023-08     127               899
240          2023-09     130              1029
241          2023-10     146              1175

=== 1. 各機種の各部品の毎月の分析 ===


/var/folders/4y/nyn48l657hx93qf7gyqvlbqr0000gn/T/ipykernel_65826/1704179757.py:64: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sales_cumsum = df_sales_aggregated.sort_values(['Area', 'Model', 'sales_year_month']).groupby(['Area', 'Model']).apply(


分析1データ形状: (2230, 20)

=== 2. 各機種の各部品の地域別分析 ===
分析2データ形状: (7375, 21)

=== 3. 各機種の各部品の製造年月別分析 ===
分析3データ形状: (12097, 17)

=== 4. 各機種の各部品の地域別かつ製造年月別分析 ===


/var/folders/4y/nyn48l657hx93qf7gyqvlbqr0000gn/T/ipykernel_65826/1704179757.py:292: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_analysis4['parts_count_zscore_12m'] = df_analysis4.groupby(['Model', 'parts_no', 'Area', 'prod_year_month']).apply(


ValueError: cannot reindex on an axis with duplicate labels

In [3]:
df_analysis4.head()

NameError: name 'df_analysis4' is not defined